In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
##GPU 사용 시
device = torch.device("cuda:1")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 한번만 선언하면 되는 애들

In [4]:
import pandas as pd
comments_unseen = pd.read_csv('../data/comments_unseen.tsv', sep = "\t")

In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
 
    def __len__(self):
        return (len(self.labels))


In [7]:
## Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 500
learning_rate =  5e-5

In [8]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [9]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 타겟으로 바꾼것으로 학습

In [10]:
dataset_train = nlp.data.TSVDataset("../data/comments_train.tsv", field_indices=[5,4], num_discard_samples=1) #5번이 타겟으로 바꾼 것
dataset_test = nlp.data.TSVDataset("../data/comments_test.tsv", field_indices=[5,4], num_discard_samples=1)
dataset_unseen = nlp.data.TSVDataset("../data/comments_unseen.tsv", field_indices=[2,1], num_discard_samples=1)

In [11]:
print(dataset_train[3], '\n', dataset_test[3], '\n', dataset_unseen[2])

['타겟이 일하는 병원 절대 안간다.', '0'] 
 ['타겟는 보호할필요가없다...가능하면 최대한의 불이익을받도록 해야한다 아...국짐같은넘들.', '0'] 
 ['타겟 사용자만 매타버스 타겟 시스템 으로 연결해도  독보적 일듯  이인원을 어떤 기업이 가져가나 지금~!  익숙한게 가장 무서운것이다', '1']


In [12]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/mglee/VSCODE/git_folder/comments_sentiment_label/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [13]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
data_unseen = BERTDataset(dataset_unseen, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
unseen_dataloader = torch.utils.data.DataLoader(data_unseen, batch_size=batch_size, num_workers=5)

for data in train_dataloader:
  print("Data: ", data)
  print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))
  break

Data:  [tensor([[   2, 1804, 6370,  ...,    1,    1,    1],
        [   2, 3257, 6005,  ...,    1,    1,    1],
        [   2,  837, 5666,  ...,    1,    1,    1],
        ...,
        [   2, 4698, 5404,  ...,    1,    1,    1],
        [   2, 2781, 7216,  ...,    1,    1,    1],
        [   2, 3437, 5507,  ..., 1844, 7139,    3]], dtype=torch.int32), tensor([18, 34,  4, 13, 11, 31, 18, 14, 25, 11, 10, 60,  9, 42, 64, 32, 64, 13,
        18, 54, 11, 59, 25, 36, 17, 13,  6, 50, 25, 32, 17, 64],
       dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0], dtype=torch.int32)]
Waveform: tensor([[   2, 1804, 6370,  ...,    1,    1,    1],
        [   2, 3257, 6005,  ...,    1,   

In [14]:
model_ourB = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model_ourB.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model_ourB.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [15]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model_ourB.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_ourB(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_ourB.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model_ourB.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_ourB(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_15215/1906873263.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/41 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7290341258049011 train acc 0.46875
epoch 1 train acc 0.7714939024390244


/tmp/ipykernel_15215/1906873263.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/7 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9285714285714286


  0%|          | 0/41 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.01533831562846899 train acc 1.0
epoch 5 train acc 0.9862804878048781


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 5 test acc 0.9330357142857143


In [16]:
torch.save(model_ourB.state_dict(), '../result/model/ourB_e5_0304.pt')  # 전체 모델 저장

ourB_model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
ourB_model.load_state_dict(torch.load('../result/model/ourB_e5_0304.pt'))
ourB_model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

### Our B TEST with Unseen

In [17]:
out_lst = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(unseen_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = ourB_model(token_ids, valid_length, segment_ids)
    out_lst.append(out.data.cpu())
    max_vals, max_indices = torch.max(out, 1)
    
pred = []
for batch in out_lst:
    for item in batch:
        pred.append(np.argmax(item.numpy()))
        
comments_unseen['pred'] = pred
print("size : ", comments_unseen.groupby('pred').size())
print("acc : ", (comments_unseen['sentiment'] == comments_unseen['pred']).mean())

100%|██████████| 53/53 [00:01<00:00, 30.58it/s]

size :  pred
0    1354
1     313
dtype: int64
acc :  0.7774445110977805


: 